# Monads
https://www.pandaandthegeek.com/post/monads-scala

In [ ]:
static int? DivBy(int a, int b) =>
    b == 0 ? default(int?) : a / b;

static int? Sample_Nullable(int a, int b) {
    var c = DivBy(a, b);
    if (c is null){
        Console.WriteLine("c is null");
        return default;
    }

    var d = DivBy(a, c.Value);
    if (d is null){
        Console.WriteLine("d is null");
        return default;
    }

    var e = DivBy(b, d.Value);
    if (e is null) {
        Console.WriteLine("e is null");
        return default;
    }

    return e;
}

Console.WriteLine($"Result: {Sample_Nullable(1, 2)}");

In [ ]:
public enum StatusCode {
    Ok,
    DivByZero
}

static (StatusCode, int) DivBy(int a, int b) =>
    b == 0 ? (StatusCode.DivByZero, default) : (StatusCode.Ok, a / b);

static (StatusCode, int) Sample_StatusCode(int a, int b) {
    var (statusC, c) = DivBy(a, b);
    
    if (statusC is StatusCode.DivByZero){
        Console.WriteLine($"e did not work {statusC}");
        return (StatusCode.DivByZero, default);    
    }

    var (statusD, d) = DivBy(a, c);
    if (statusD is StatusCode.DivByZero) {
        Console.WriteLine($"e did not work {statusD}");
        return (StatusCode.DivByZero, default);    
    }

    var (statusE, e) = DivBy(a, c);
    if (statusE is StatusCode.DivByZero) {
        Console.WriteLine($"e did not work {statusE}");
        return (StatusCode.DivByZero, default);    
    }

    return (statusE, e);
}

Console.WriteLine($"Result: {Sample_StatusCode(1, 2)}");

In [ ]:
readonly struct Result<A>
{
    private readonly A value;
    private readonly bool isOk;
    private readonly string error = string.Empty;

    private Result(A value)
    {
        this.value = value;
        this.isOk = true;
    }

    private Result(string error)
    {
        this.error = error;
        this.isOk = false;
    }

    public static Result<A> Identity(A value) => new(value);

    public static Result<A> Error(string error) => new(error);

    public Result<B> Bind<B>(Func<A, Result<B>> transform)
    {
        if (isOk)
        {
            Console.WriteLine($"Bind: {this}");
            return transform(value);
        }
        else
        {
            Console.WriteLine($"Skip bind: {this}");
            return Result<B>.Error(error);
        }
    }

    public override string ToString() =>
        isOk
            ? $"Ok({value})"
            : $"Error({error})";
}

static Result<int> DivBy(int a, int b) =>
    b == 0
        ? Result<int>.Error("Cannot divide by zero")
        : Result<int>.Identity(a / b);

static Result<int> Sample_Result(int a, int b)
{
    var cResult = DivBy(a, b);
    var dResult = cResult.Bind(c => DivBy(a, c));
    var eResult = dResult.Bind(d => DivBy(b, d));
    return eResult;
}

Console.WriteLine($"Result: {Sample_Result(1, 2)}");

In [ ]:
public class Writer<T, L>
{
    private readonly T value;
    private IEnumerable<L> Elements { get; }
    
    public (T, IEnumerable<L>) Unsafe() => (value, Elements);

    public static Writer<T, L> Identity(T value, L element) => Identity(value, new[] {element});
    public static Writer<T, L> Identity(T value, IEnumerable<L> elements) => new Writer<T, L>(value, elements);

    private Writer(T value, IEnumerable<L> elements)
    {
        this.value = value;
        this.Elements = elements;
    }
    
    public Writer<R, L> Map<R>(Func<T, R> fn)
    {
        return FlapMap(v => Writer<R, L>.Identity(fn(v), Elements));
    }
    
    public Writer<R, L> FlapMap<R>(Func<T, Writer<R, L>> fn)
    {
        var (v, newElements) = fn(value).Unsafe();
        
        var acc = new List<L>(Elements);
        acc.AddRange(newElements);
        return new Writer<R, L>(v, acc);
    }
}

public static class Calculator
{
    public static Writer<int, string> sum(int a, int b) =>
        Writer<int, string>.Identity(a + b, "sum (" + a + " + " + b + ")");

    public static Writer<int, string> mul(int a, int b) =>
        Writer<int, string>.Identity(a * b, "mul (" + a + " * " + b + ")");
        
    public static Writer<int, string> dif(int a, int b) =>
        Writer<int, string>.Identity(a - b, "diff (" + a + " - " + b + ")");
        
    public static Writer<int, string> squared(int a) =>
        Writer<int, string>.Identity(a * a, "squared (" + a + ")");
}

var final = Calculator
    .sum(5, 5)
    .FlapMap(n => Calculator.mul(n, 2))
    .FlapMap(n => Calculator.dif(n, 18))
    .FlapMap(n => Calculator.squared(n));
    
var (value, log) = final.Unsafe();

Console.WriteLine($"Calculator result: {value}");

foreach (string s in log)
{
    Console.Write(s + " ");
    Console.WriteLine();
}